# Strong Scaling Study

## Example: Compute $\pi$

### Serial Version

In [1]:
def pi_serial(n):
    import numpy as np
    np.random.seed(0)
    x = np.random.uniform(-1, 1, n)
    y = np.random.uniform(-1, 1, n)
    f_hat = np.where(x**2 + y**2 <= 1., 1, 0)
    V = 4.0
    return V*np.sum(f_hat)/n

In [19]:
N = 10000000
pi_serial(N)

3.1412900000000001

In [20]:
t_serial = %timeit -o pi_serial(N)

458 ms ± 3.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
t_serial.average

0.45840940771397853

### Parallel Version

In [7]:
from ipyparallel import Client
rc = Client()
rc.ids
dview = rc[:]
dview.scatter('id', rc.ids, block=True)

In [9]:
%px print(id)

[stdout:0] [0]
[stdout:1] [1]
[stdout:2] [2]
[stdout:3] [3]


In [22]:
def pi_serial_seed(n):
    import numpy as np
    np.random.seed(id)
    x = np.random.uniform(-1, 1, n)
    y = np.random.uniform(-1, 1, n)
    f_hat = np.where(x**2 + y**2 <= 1., 1, 0)
    V = 4.0
    return V*np.sum(f_hat)/n

In [24]:
def pi_parallel(N, rc):
    dview = rc[:]
    dview.scatter('id', rc.ids, block=True)
    nengines = len(rc.ids)
    work = [10000000//nengines]*nengines
    results = dview.map_sync(pi_serial_seed, work)
    return sum(results)/nengines

In [32]:
pi_parallel(N, rc[:2])

TypeError: names must be strs, not slice(None, None, None)

In [30]:
nengines = len(rc.ids)
t_parallel = np.zeros(nengines+1)
for i in arange(2,nengines+1):
t_parallel[i] = %timeit -o pi_parallel(N, rc)

IndentationError: expected an indented block (<ipython-input-30-4118e197b416>, line 4)

In [31]:
?rc